In [193]:
import pickle
import json
import random
import tensorflow
import tflearn
import numpy
import nltk

In [195]:
with open("intents.json", 'rb') as file:
    data = json.load(file)

In [196]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'Hello',
    'Hola',
    'Hey',
    'Hi',
    'Hello',
    'Good morning',
    'Good Evening'],
   'responses': ['Hello, Welcome to Our IT Chatbot',
    'Good to see you, Welcome to our IT Chatbot',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time',
    'Bye Chatbot'],
   'responses': ['See you!',
    'Have a nice day',
    'Bye! Come back again soon.',
    'Happy to help'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help! Any other issues?',
    'Any time! Any other issues I can help with?',
    'My pleasure! Any other Issues I can help with?'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': ['q', 'random'],
   'responses': ["Sorry, 

In [197]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

In [198]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [199]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

In [200]:
labels = sorted(labels)

In [201]:
training = []
output = []

In [202]:
out_empty = [0 for _ in range(len(labels))]
for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

In [203]:
training = numpy.array(training)
output = numpy.array(output)

In [210]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 128)
net = tflearn.fully_connected(net, 128)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

In [211]:
model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=32, show_metric=True)
#model.save("model.tflearn")

Training Step: 2999  | total loss: 0.43419 | time: 0.047s
| Adam | epoch: 1000 | loss: 0.43419 - acc: 0.9259 -- iter: 64/88
Training Step: 3000  | total loss: 0.39834 | time: 0.062s
| Adam | epoch: 1000 | loss: 0.39834 - acc: 0.9333 -- iter: 88/88
--


In [212]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [213]:
def chat():
    print("Comece a conversar com o bot (digite quit para parar)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] > 0.7:
            for tg in data["intents"]:
                if tg["tag"] == tag:
                    responses = tg["responses"]

            print(random.choice(responses))
        else:
            print("I didn't get that, try again. ")

In [ ]:
chat()

Comece a conversar com o bot (digite quit para parar)!
You: Hi
Hi there, how can I help?
You: i have problem usb
I didn't get that, try again. 
You:  I think I got a virus
A detailed step by step guide to remove the virus from the computer has been provided on the following link: 
https://www.easeus.com/file-recovery/remove-virus-without-antivirus.html 
and
https://www.pcworld.com/article/243818/how-to-remove-malware-from-your-windows-pc.html
If the issues are still there, consult the IT team
You: USB
First things to check are:
1) Does the device work in a different USB port on the machine?
2) Are other devices recognised in that port?
3) Does the device work on another user’s machine?
 If you have tried these troubleshooting methods and still no luck, then your IT support help-desk can proceed with some more in-depth troubleshooting.
You: 1
I didn't get that, try again. 
You: Does the device work in a different USB port on the machine?
First things to check are:
1) Does the device wor